In [1]:
import pandas as pd
from datetime import timedelta
import glob
import pyodbc

##  Add Trip_id

In [3]:
def add_trip_ids(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime').reset_index(drop=True)
    
    # Calculate time difference between consecutive rows
    time_diff = df['datetime'].diff()
    
    # New trip starts when time difference > 30 minutes (and for first record)
    new_trip = (time_diff > timedelta(minutes=30)) | (time_diff.isna())
    
    
    df['trip_id'] = new_trip.cumsum()
    
    return df


def process_taxi_file(file_path):

    df = pd.read_csv(file_path, 
                     names=['taxi_id', 'datetime', 'longitude', 'latitude'])
    
    initial_rows = len(df)
    df = df.drop_duplicates()
    dropped_rows = initial_rows - len(df)
    
    if dropped_rows > 0:
        print(f"Removed {dropped_rows} duplicate rows from {file_path}")
        
 
    df_with_trips = add_trip_ids(df)
    
    return df_with_trips

## Remove Duplicates

In [4]:
result_df = pd.DataFrame()

for file_path in glob.glob('C:\\priya_hari\\UW Tacoma\\MSCSS\\Fourth Quarter\\TCSS_565\\Trajectory_Dataset\\*.txt'):
    df = process_taxi_file(file_path)
    result_df = pd.concat([result_df, df], ignore_index=True)

Removed 24 duplicate rows from C:\priya_hari\UW Tacoma\MSCSS\Fourth Quarter\TCSS_565\Trajectory_Dataset\1.txt
Removed 219 duplicate rows from C:\priya_hari\UW Tacoma\MSCSS\Fourth Quarter\TCSS_565\Trajectory_Dataset\10.txt
Removed 51 duplicate rows from C:\priya_hari\UW Tacoma\MSCSS\Fourth Quarter\TCSS_565\Trajectory_Dataset\100.txt


In [ ]:
result_df.to_csv('processed_dataset.csv', index=False)

In [7]:
result_df.dtypes

taxi_id               int64
datetime     datetime64[ns]
longitude           float64
latitude            float64
trip_id               int64
dtype: object

## Check for null values

In [8]:
null_counts = result_df.isnull().sum()
null_counts

taxi_id      0
datetime     0
longitude    0
latitude     0
trip_id      0
dtype: int64

## Establishing Connection and 

In [9]:
def insert_taxi_data(df, server, database):
    conn_str = (
        'Driver={ODBC Driver 18 for SQL Server};'
        f'Server={server};'
        f'Database={database};'
        'Trusted_Connection=yes;'
        'TrustServerCertificate=yes;'
    )
    
    try:
        # Create connection
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        print("Connected to the database")
        
        rows_processed = 0
        
        # SQL insert statement using geography::Point
        insert_query = """
        INSERT INTO taxi_trips 
            (taxi_id, trip_id, datetime, longitude, latitude, location)
        VALUES 
            (?, ?, ?, ?, ?, geography::Point(?, ?, 4326))
        """
        
        for _, row in df.iterrows():
            try:
                cursor.execute(
                    insert_query,
                    int(row['taxi_id']),
                    int(row['trip_id']),
                    row['datetime'],
                    float(row['longitude']),
                    float(row['latitude']),
                    float(row['latitude']),    # Point takes latitude first
                    float(row['longitude'])     # then longitude
                )
                
                rows_processed += 1
                if rows_processed % 100 == 0:  # Print progress every 100 rows
                    print(f"Processed {rows_processed} rows")
                
            except Exception as e:
                print(f"Error inserting row {row['taxi_id']}, {row['trip_id']}, {row['datetime']}: {str(e)}")
                raise
                
        # Commit the transaction and close connections
        conn.commit()
        cursor.close()
        conn.close()
        print(f"Data insertion completed successfully. Total rows processed: {rows_processed}")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        raise

In [10]:
if __name__ == "__main__":
    server = 'RIYA_SURFACE\\PRIYAMSSQL'
    database = 'TCSS565_TrajectoryDB'
    
        
        # Then try the insert
insert_taxi_data(result_df, server, database)
        

Successfully connected to the database
Processed 100 rows
Processed 200 rows
Processed 300 rows
Processed 400 rows
Processed 500 rows
Processed 600 rows
Processed 700 rows
Processed 800 rows
Processed 900 rows
Processed 1000 rows
Processed 1100 rows
Processed 1200 rows
Processed 1300 rows
Processed 1400 rows
Processed 1500 rows
Processed 1600 rows
Processed 1700 rows
Processed 1800 rows
Processed 1900 rows
Processed 2000 rows
Processed 2100 rows
Processed 2200 rows
Processed 2300 rows
Processed 2400 rows
Processed 2500 rows
Processed 2600 rows
Processed 2700 rows
Processed 2800 rows
Processed 2900 rows
Processed 3000 rows
Processed 3100 rows
Processed 3200 rows
Processed 3300 rows
Processed 3400 rows
Processed 3500 rows
Processed 3600 rows
Processed 3700 rows
Processed 3800 rows
Processed 3900 rows
Processed 4000 rows
Processed 4100 rows
Processed 4200 rows
Processed 4300 rows
Processed 4400 rows
Processed 4500 rows
Processed 4600 rows
Processed 4700 rows
Processed 4800 rows
Processed 